In [1]:
import pandas as pd
import json
from math import radians, cos, sin, asin, sqrt
import math
import numpy as np
from geopy.geocoders import Nominatim

In [2]:
# import the data in a dataframe
json_data=open('nuclear_tests.json').read()
data = json.loads(json_data)
df = pd.DataFrame(data)
df.head()
print(len(df))

2065


In [3]:
print(str(len(df[pd.isnull(df['decimal latitude/longitude'])])) + ' / ' + str(len(df)) + ' entries do not have coordinates')
print(str(len(df[df['site'] == ''])) + ' / ' + str(len(df)) + ' entries do not have a site')
distinct_sites = set(df['site'])
print('There are ' + str(len(distinct_sites)) + ' distinct sites')

1211 / 2065 entries do not have coordinates
119 / 2065 entries do not have a site
There are 36 distinct sites


In [4]:
# Only one entry has neither a site nor coordinates:
df[(pd.isnull(df['decimal latitude/longitude'])) & (df['site'] == '')]

,body wave magnitude,country,date,decimal latitude/longitude,name,original time,site,type,yield
2064,NaN,UNK,22/09/1979,NaN,-,03:00:00 (Universal Coordinated Time),,Atmospheric,NaN


In [5]:
# remove null site/location from dataframe
df = df[(pd.notnull(df['decimal latitude/longitude'])) | (df['site'] != '')]

In [6]:
# We group the dataframe by sites and export the coordinates of each event in a distinct list for each site
# the goal is to see events on a same site can have very different coordinates or not.
coord_lists = df.fillna('-').groupby('site')['decimal latitude/longitude'].apply(list)

In [7]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [8]:
no_coord_sites = []
coord_sites = dict()
for i in range(0, len(coord_lists)):
    dists = []
    lats = []
    longs = []
    lat0 = ""
    long0 = ""
    print(coord_lists.index[i])
    for event in coord_lists[i]:
        if('/' in str(event)):
            lat0, long0 = event.split('/')
            break
    if (lat0!="" and long0!=""):
        for event in coord_lists[i]:
            event = str(event)
            if('/' in event):
                lat, long = event.split('/')
                dists.append(haversine(float(long0), float(lat0), float(long), float(lat)))
                lats.append(float(lat))
                longs.append(float(long))
        
        print(np.mean(dists))
        meanlat = np.mean(lats)
        meanlong = np.mean(longs)
        coord_sites[coord_lists.index[i]]=str(meanlat)+"/"+str(meanlong)
    else:
        no_coord_sites.append(coord_lists.index[i])
        print("No distances")
    print("---------")



1651.56727716
---------
Amchitka, Alaska
4.78866535241
---------
Bikini Is (USA)
32.7202064536
---------
Carlsnad, New Mexico (USA)
No distances
---------
Central Nevada
No distances
---------
Christmas Island
No distances
---------
Christmas Island (USA)
No distances
---------
Ekker, Algeria
1.42693253151
---------
Emu Fields, Australia
No distances
---------
Enewetak (USA)
53.7383584401
---------
Fallon, Nevada
No distances
---------
Farmington, New Mexico
No distances
---------
Grand Valley, Colorado
No distances
---------
Hattiesburg, Mississippi
No distances
---------
Hiroshima, Japan
0.0
---------
Johnston Island
No distances
---------
Kharan
0.0
---------
Lop Nor
54.0866798425
---------
Maralinga
No distances
---------
Missile Testing Range
138.40896036
---------
Monte Bello Is
No distances
---------
NTS
No distances
---------
Nagasaki, Japan
0.0
---------
Nellis Air Force Range
No distances
---------
Nevada Test Site
5.55569994183
---------
New Mexico (USA)
No distances
------

In [9]:
# As we can see, some events have a site but no location. 

In [10]:
#first make a dict site => coords (from geopy)
geolocator = Nominatim()
geopy_site_coords = dict()
for site in no_coord_sites:
    location = geolocator.geocode(site)
    if location is not None:
        coord = str(location.latitude) + "/" + str(location.longitude)
        geopy_site_coords[site] = coord
    else:
        print("coordinates not found for site " + str(site))
#site_coords

coordinates not found for site Carlsnad, New Mexico (USA)
coordinates not found for site Emu Fields, Australia
coordinates not found for site Nellis Air Force Range


In [11]:
print(len(no_coord_sites))
print(len(geopy_site_coords))

19
16


In [12]:
# For the 3 other position we have to do it by hand (using https://www.latlong.net/)
geopy_site_coords['Carlsnad, New Mexico (USA)']="32.420674/-104.228837"
geopy_site_coords['Emu Fields, Australia']="-28.698333/132.371389"
geopy_site_coords['Nellis Air Force Range']="36.308119/-115.049367"

In [13]:
for i in df.index:
    if('/' not in str(df['decimal latitude/longitude'][i])):
        if (df['site'][i] in geopy_site_coords.keys()):
            df['decimal latitude/longitude'][i] = geopy_site_coords[df['site'][i]]

In [14]:
print(str(len(df[pd.isnull(df['decimal latitude/longitude'])])) + ' / ' + str(len(df)) + ' entries do not have coordinates')

927 / 2064 entries do not have coordinates


In [15]:
df[pd.isnull(df['decimal latitude/longitude'])].head()

,body wave magnitude,country,date,decimal latitude/longitude,name,original time,site,type,yield
1008,4.4,USA,23/09/1992,NaN,DIVIDER,15:04:00 (Universal Coordinated Time),Nevada Test Site,Underground,NaN
1009,4.4,USA,18/09/1992,NaN,HUNTERS TROPHY,17:00:00 (Universal Coordinated Time),Nevada Test Site,Underground,NaN
1010,NaN,USA,23/06/1992,NaN,GALENA-YELLOW,15:00:00 (Universal Coordinated Time),Nevada Test Site,Underground,NaN
1011,NaN,USA,19/06/1992,NaN,VICTORIA,16:45:00 (Universal Coordinated Time),Nevada Test Site,Underground,NaN
1012,NaN,USA,30/04/1992,NaN,DIAMOND FORTUNE,16:30:00 (Universal Coordinated Time),Nevada Test Site,Underground,NaN


In [16]:
# Now we need to use the informations we get for each site in our data and use it to extrapolate the location of the 
for i in df.index:
    if('/' not in str(df['decimal latitude/longitude'][i])):
        if (df['site'][i] in coord_sites.keys()):
            df['decimal latitude/longitude'][i] = coord_sites[df['site'][i]]

In [17]:
print(str(len(df[pd.isnull(df['decimal latitude/longitude'])])) + ' / ' + str(len(df)) + ' entries do not have coordinates')

0 / 2064 entries do not have coordinates


In [18]:
# We now have coordinates for every item in our dataframe
df[pd.isnull(df['decimal latitude/longitude'])].head()

,body wave magnitude,country,date,decimal latitude/longitude,name,original time,site,type,yield


In [19]:
df_selected = df[['body wave magnitude', 'country', 'date', 'decimal latitude/longitude', 'name', 'site', 'type']]

In [20]:
df_selected['lat'], df_selected['lng'] = df['decimal latitude/longitude'].str.split('/', 1).str

C:\Users\Alain\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_selected.drop('decimal latitude/longitude', axis=1, inplace=True)

In [22]:
df_selected.columns = ['magnitude', 'country', 'date', 'name', 'site', 'type', 'lat','lng']

In [23]:
df_selected.head()

,magnitude,country,date,name,site,type,lat,lng
0,4.9,CHN,29/07/1996,-,Lop Nor,Underground,41.82,88.42
1,5.9,CHN,08/06/1996,-,Lop Nor,Underground,41.66,88.69
2,6,CHN,17/08/1995,-,Lop Nor,Underground,41.56,88.8
3,6.1,CHN,15/05/1995,-,Lop Nor,Underground,41.6,88.82
4,6,CHN,07/10/1994,-,Lop Nor,Underground,41.66,88.75


In [24]:
df_selected.to_csv("MapTimeline/data/nuclear_test.tsv", sep='\t')

In [25]:
results = {}
for key, df_gb in df.iterrows():
    results[str(key)] = df_gb.to_dict()
import json
with open('MapTimeline/data/nuclear_test_cleaned.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)